# Confidence Analysis

In [0]:
%sql
-- Set catalog and schema context for all operations
USE CATALOG users;
USE SCHEMA luis_herrera;

-- Set execution timeout to 4 hours
SET spark.databricks.execution.timeout = 14400;
SET spark.sql.shuffle.partitions =64;

In [0]:
%sql
-- Load the core usecase curated table as a temporary view
CREATE OR REPLACE TABLE full_usecase_curated
AS SELECT * FROM main.gtm_data.core_usecase_curated;

-- Display the first 10 lines
SELECT * FROM full_usecase_curated LIMIT 10;

Regional Distribution for Parallel Processing:
============================================================
AMER                 | AMER Industries                | 2,166 records
AMER                 | AMER Enterprise & Emerging     | 1,964 records
Unknown              | EMEA                           | 1,942 records
APJ                  | APJ                            | 865 records
Unknown              | AMER Industries                | 842 records
EMEA                 | EMEA                           | 608 records
LATAM                | AMER Enterprise & Emerging     | 246 records
Unknown              | APJ                            | 179 records
Unknown              | AMER Enterprise & Emerging     | 79 records
Unknown              | Unknown                        | 6 records
EMEA                 | Unknown                        | 2 records
============================================================
TOTAL                |                                | 8,899 records

Will create 11 parallel processing partitions
Largest partitions (potential bottlenecks):
  amer_amer_industries: 2,166 records
  amer_amer_enterprise_and_emerging: 1,964 records
  null_emea: 1,942 records
  apj_apj: 865 records
  null_amer_industries: 842 records

In [0]:
%sql
CREATE OR REPLACE TABLE  filtered_usecases
AS SELECT 
  * EXCEPT (field_manager, account_solution_architect, sales_region, num_of_blockers, sales_manager),
  CASE 
    WHEN field_manager IS NULL THEN 'FLM not assigned'
    ELSE field_manager
  END AS field_manager,
  CASE 
    WHEN account_solution_architect IS NULL THEN 'SA not assigned'
    ELSE account_solution_architect
  END AS account_solution_architect,
  CASE
    WHEN sales_region IS NOT NULL THEN sales_region
    WHEN sales_region IS NULL AND business_unit IS NOT NULL THEN business_unit
    WHEN sales_region IS NULL AND business_unit IS NULL THEN 'No assigned'
    ELSE 'No assigned'
  END AS sales_region,
  COALESCE(num_of_blockers, 0) AS num_of_blockers,
  -- Add sales_manager logic
  CASE
    WHEN sales_manager IS NULL THEN 'Sales Manager not assigned'
    ELSE sales_manager
  END AS sales_manager,
  -- Logical field to detect stale use cases
  CASE 
    WHEN last_modified_date IS NOT NULL AND last_modified_date < DATE_SUB(CURRENT_DATE(), 14) THEN true
    ELSE false
  END AS is_stale
FROM full_usecase_curated
WHERE 
  estimated_monthly_dollar_dbus >= 1000
  AND stage NOT IN ('U1', 'Lost', 'Closed', 'Disqualified')
--  AND usecase_name NOT LIKE '%Unity Catalog%'
--  AND usecase_name NOT LIKE '%UC%'
--  AND usecase_name NOT LIKE '%Governance%'
  AND target_live_date BETWEEN CURRENT_DATE() AND ADD_MONTHS(CURRENT_DATE(), 6)
  AND last_modified_date >= ADD_MONTHS(CURRENT_DATE(), -6)
--  AND (sales_region = 'EMEA' OR  business_unit = 'EMEA')
ORDER BY estimated_monthly_dollar_dbus DESC;

-- Print the first row of the filtered_usecases view
SELECT * FROM filtered_usecases ORDER BY estimated_monthly_dollar_dbus  ASC LIMIT 10;

In [0]:
%sql
-- Drop the context_prompt variable if it exists before creating
DROP TEMPORARY VARIABLE IF EXISTS context_prompt;

-- Define the context summarization prompt as a variable
DECLARE VARIABLE context_prompt STRING DEFAULT 
  'Summarize the following context and business impact into a concise paragraph (max 4 sentences). Focus on the main challenge, expected business value, the main type of contact (e.g., Executive sponsor, technical decision maker, Economic decision maker, etc), and whether funding is available or approved. Context: ';

-- Generate contextual summaries using AI (optimized for clarity, null safety, and token cost)
CREATE OR REPLACE TABLE filtered_usecases_with_context
AS SELECT 
  *,
  CASE 
    WHEN (sdr_bdr_notes IS NOT NULL AND LENGTH(TRIM(sdr_bdr_notes)) > 0)
      OR (business_impact IS NOT NULL AND LENGTH(TRIM(business_impact)) > 0)
      OR (business_outcome_description IS NOT NULL AND LENGTH(TRIM(business_outcome_description)) > 0)
      OR (ae_authority IS NOT NULL AND LENGTH(TRIM(ae_authority)) > 0)
      OR (ae_money_budget IS NOT NULL AND LENGTH(TRIM(ae_money_budget)) > 0)
    THEN
      ai_summarize(
        CONCAT(
          context_prompt,
          COALESCE(TRIM(LEFT(regexp_replace(sdr_bdr_notes, '<[^>]+>', ''), 2000)), 'No context provided. '),
          'Business Impact: ', COALESCE(TRIM(business_impact), 'No business impact provided. '),
          'Business Outcome: ', COALESCE(TRIM(business_outcome_description), 'No business outcome provided. '),
          'Main Contact Type: ', COALESCE(TRIM(ae_authority), 'Not specified.'),
          'Funding Status: ', COALESCE(TRIM(ae_money_budget), 'Not specified.')
        ),
        100
      )
    ELSE
      'No context or business impact provided.'
  END AS context
FROM filtered_usecases;

-- Simple test: Show the context for the first rows
SELECT context
FROM filtered_usecases_with_context
LIMIT 10;

In [0]:
%sql
-- Generate one-liner summaries using AI (optimized for clarity and efficiency)
CREATE OR REPLACE TABLE usecases_with_oneliner
AS SELECT 
  *,
  CASE
    WHEN (usecase_name IS NOT NULL AND LENGTH(TRIM(usecase_name)) > 0)
      OR (usecase_description IS NOT NULL AND LENGTH(TRIM(usecase_description)) > 0)
    THEN
      ai_summarize(
        CONCAT(
          'Generate a short, clear one-line summary (max 50 words) describing the core business objective of this use case. ',
          'Account Contex:', COALESCE(TRIM(context), 'No account context'),
          'Use Case Name: ', COALESCE(TRIM(usecase_name), 'Use Case name not provided.'),
          '. Description: ', COALESCE(TRIM(usecase_description), 'No description provided.')
        ),
        50
      )
    ELSE
      'No use case name or description provided'
  END AS one_liner
FROM filtered_usecases_with_context;

-- Simple test: Show the one_liner for the first row
SELECT one_liner
FROM usecases_with_oneliner
LIMIT 10;

In [0]:
%sql
-- Generate next steps summaries using AI (optimized for clarity, cost, and reliability)
CREATE OR REPLACE TABLE usecases_with_nextsteps
AS SELECT 
  *,
  CASE 
    WHEN demand_plan_stage_next_steps IS NOT NULL 
         AND LENGTH(TRIM(demand_plan_stage_next_steps)) > 0 THEN
      ai_summarize(
        CONCAT(
          'Summarize the following next steps into a short, action-oriented sentence focusing on immediate priorities: ',
          TRIM(demand_plan_stage_next_steps)
        ),
        100
      )
    ELSE
      'No next steps provided'
  END AS next_steps_summary
FROM usecases_with_oneliner;

-- Simple test: Show the next_steps_summary for the first 10 rows
SELECT next_steps_summary
FROM usecases_with_nextsteps
LIMIT 10;

In [0]:
%sql
-- Generate implementation notes summaries using AI
CREATE OR REPLACE TABLE usecases_with_implementation
AS SELECT 
  *,
  CASE 
    WHEN implementation_notes IS NOT NULL AND LENGTH(TRIM(implementation_notes)) > 0 THEN
      ai_summarize(
        CONCAT(
          'Summarize the following implementation notes into a short, action-oriented paragraph focusing on key steps, milestones, and ownership: ',
          implementation_notes
        ),
        100
      )
    ELSE
      'There is no implementation plan or not specified in Salesforce'
  END AS implementation_notes_summary
FROM usecases_with_nextsteps;

-- Simple test: Show the implementation_notes_summary for the first row
SELECT implementation_notes_summary
FROM usecases_with_implementation
LIMIT 10;

In [0]:
%sql
-- Classify use case types using AI
CREATE OR REPLACE TABLE usecases_with_types
AS SELECT 
  *,
  CASE
    WHEN (usecase_name IS NOT NULL AND LENGTH(TRIM(usecase_name)) > 0)
      OR (usecase_description IS NOT NULL AND LENGTH(TRIM(usecase_description)) > 0)
    THEN COALESCE(
      ai_classify(
        concat('Usecase Name: ', coalesce(trim(usecase_name), '.'), ' Description: ', coalesce(trim(usecase_description), '.')),
        array(
          'Machine Learning', 'Generative AI', 'Data Warehousing', 'Migration AI', 'Migration Unity Catalog',
          'Migration DWH', 'Migration ETL', 'Migration Streaming', 'Migration BI', 
          'BI', 'ETL', 'Governance', 'Streaming', 'Ingestion', 'Platform', 'Other'
        )
      ),
      'Other'
    )
    ELSE 'Other'
  END AS usecase_type
FROM usecases_with_implementation;

-- Show only the created field
SELECT usecase_type FROM usecases_with_types LIMIT 10;

In [0]:
%sql
-- Drop the business_usecase_prompt variable if it exists before creating
DROP TEMPORARY VARIABLE IF EXISTS business_usecase_prompt;

-- Define the business use case summarization prompt as a variable
DECLARE VARIABLE business_usecase_prompt STRING DEFAULT 
  'Given a use case name, description and industry imperativ, return ONLY a concise business use case label (e.g. Fraud Detection, Inventory Forecasting, Customer Segmentation). Output exactly one short phrase, max 5 words, no punctuation or explanation. Usecase Name: ';

CREATE OR REPLACE TABLE usecases_with_business_types
AS SELECT 
  *,
  COALESCE(
    ai_query(
      'databricks-gemma-3-12b',
      CONCAT(
        business_usecase_prompt,
        COALESCE(TRIM(usecase_name), 'Unknown'), 
        '. Description: ', COALESCE(TRIM(usecase_description), 'No description'),
        '. Industry Imperative: ', COALESCE(TRIM(industry_imperative), 'No industry imperative')
      ),
      named_struct('max_tokens', 20, 'temperature', 0.0, 'failOnError', false)
    ),
    'Unknown Business Use Case'
  ) AS business_usecase_type
FROM usecases_with_types;

-- Simple test: Show the business_usecase_type for the first row
SELECT business_usecase_type
FROM usecases_with_business_types
LIMIT 10;

In [0]:
CREATE OR REPLACE TABLE usecases_with_business_types AS
SELECT
  *,
  COALESCE(
    ai_gen(
      CONCAT(
        'You are the Value Soundbite Agent. Your task is to generate a single elevator pitch sentence that connects a technical solution to a clear business outcome. You need to extract from the context\n\n',
        'Customer name: ', COALESCE(account_name, 'Unknown'), '\n',
        'Technical Challenge (current pain/limitation): ', COALESCE(one_liner, 'Not specified'), '\n',
        'Technical Solution / Benefit (what the technology enables): ', COALESCE(usecase_type, 'Not specified'), '\n',
        'Business Benefit (how analysts/users/teams work better, faster, cheaper): ', COALESCE(business_usecase_type, 'Not specified'), '\n',
        'Business Goal (strategic outcome such as cost savings, revenue growth, margin improvement, risk reduction, etc.): ', COALESCE(context, 'Not specified'), '\n\n',
        'Output:\n',
        'Write one concise, executive-ready sentence using this formula:\n',
        'By [implementing the solution] to tackle [technical challenge], [Customer] will [achieve business benefit] and [achieve business goal], enabling [strategic initiative/program] which will [ultimate business outcome].\n\n',
        'Guidelines:\n',
        'Keep the sentence short, punchy, and outcome-driven.\n',
        'Focus on benefits, not features.\n',
        'Use business-friendly language (no jargon).'
      )
    ),
    'No soundbyte generated'
  ) AS soundbyte
FROM usecases_with_business_types;

SELECT soundbyte
FROM usecases_with_business_types
LIMIT 10;

In [0]:
%sql
-- Drop the blockers_prompt variable if it exists before creating
DROP TEMPORARY VARIABLE IF EXISTS blockers_prompt;

-- Define the blockers summarization prompt as a variable
DECLARE VARIABLE blockers_prompt STRING DEFAULT 'There are {num} blockers. Briefly summarize the main theme or issue based on the latest blocker comment: ';

CREATE OR REPLACE TABLE usecases_with_blockers
AS SELECT *,
       CASE 
         WHEN COALESCE(num_of_blockers, 0) = 0 THEN 'There are 0 blockers'
         WHEN (last_modified_blocker_category IS NULL OR LENGTH(TRIM(last_modified_blocker_category)) = 0)
           AND (last_modified_blocker_comment IS NULL OR LENGTH(TRIM(last_modified_blocker_comment)) = 0)
           THEN 'Blocker details not provided'
         ELSE CONCAT(
           'There are ', COALESCE(CAST(num_of_blockers AS STRING), '0'), ' blockers. ',
           ai_summarize(
             CONCAT(
               REPLACE(blockers_prompt, '{num}', COALESCE(CAST(num_of_blockers AS STRING), '0')),
               CASE 
                 WHEN last_modified_blocker_category IS NOT NULL AND LENGTH(TRIM(last_modified_blocker_category)) > 0 
                   THEN CONCAT('Category: ', TRIM(last_modified_blocker_category), '. ')
                 ELSE ''
               END,
               COALESCE(TRIM(last_modified_blocker_comment), 'No blocker comment provided.')
             ),
             50
           )
         )
       END AS blockers_summary
FROM usecases_with_business_types;

-- Simple test: Show the blockers_summary for the first row
SELECT blockers_summary
FROM usecases_with_blockers
LIMIT 10;

In [0]:
%sql
-- Drop the ai_confidence_prompt variable if it exists before creating
DROP TEMPORARY VARIABLE IF EXISTS ai_confidence_prompt;

-- Define the AI confidence prompt as a variable (using \n for clarity and structure)
DECLARE VARIABLE ai_confidence_prompt STRING DEFAULT 
  'Using the MEDDPICC framework, and the following context, output a confidence score (0-100) for successful go-live. Only output the number that is multiple of 10.\n'
 'Context:\n'
 'Account context: {context}\n'
 'Use Case: {one_liner}\n'
 'Next steps: {next_steps_summary}\n'
 'Implementation notes: {implementation_notes_summary}\n'
 'Blockers: {blockers_summary}\n'
 'Competition: {competition_string}\n'
 'Timeline Urgency: {timeline_urgency}\n';

-- Define the advanced AI confidence prompt as a variable
DROP TEMPORARY VARIABLE IF EXISTS ai_confidence_prompt_advanced;

DECLARE VARIABLE ai_confidence_prompt_advanced STRING DEFAULT
  'You are a Databricks Solutions Architect evaluating a customer use case and scoring its likelihood to reach production and generate sustained usage.\n\n'

  'Rate the use case using these MEDDPICC-based dimensions (0–10 each):\n'
  '1. Pain – Clear, urgent business problem?\n'
  '2. Champion – Identified internal advocate?\n'
  '3. Implementation Plan – Concrete next steps and ownership?\n'
  '4. Decision Process – Clear decision path?\n'
  '5. Urgency – Time pressure or deadline?\n'
  '6. Competition Awareness – Known competitors?\n'
  '7. Measurable Impact – Defined success metrics?\n'
  '8. Major Blockers – Significant obstacles (higher score = **more blockers**).\n\n'

'Rules:\n'
'- Base scores strictly on available evidence — do not speculate or imagine missing info.\n'
'- If information is missing, assign a low score (≤3) for that dimension.\n'
'- Use these weightings: Pain 25%, Champion 20%, Implementation Plan 20%, Decision Process 10%, Urgency 10%, Competition Awareness 5%, Measurable Impact 2%, Major Blockers -8% (deductive).\n'
'- Compute the total confidence_score as: (Pain×0.25 + Champion×0.20 + ImplementationPlan×0.20 + DecisionProcess×0.10 + Urgency×0.10 + CompetitionAwareness×0.05 + MeasurableImpact×0.02 - MajorBlockers×0.08) × 10.\n'
'- A perfect use case (10s across positive dimensions, 0 blockers) scores 100.\n'
'- The total confidence_score must be between 0 and 100, rounded to the nearest multiple of 10.\n'


  'Context:\n'
  'Account context: {context}\n'
  'Use Case: {one_liner}\n'
  'Next steps: {next_steps_summary}\n'
  'Implementation notes: {implementation_notes_summary}\n'
  'Blockers: {blockers_summary}\n'
  'Competition: {competition_string}\n'
  'Timeline Urgency: {timeline_urgency}\n\n'

  'Return output in **this exact format** (no text before or after):\n\n'
  'CONFIDENCE_SCORE: [0–100 number]\n'
  'LEVEL: [High || Medium || Low]\n'
  'RATIONALE: [Brief explanation, 2–3 sentences max]\n'
  'DIMENSION_SCORES: Pain=x, Champion=x, ImplementationPlan=x, DecisionProcess=x, Urgency=x, CompetitionAwareness=x, MajorBlockers=x, MeasurableImpact=x';

-- Create or replace a temporary view 'usecases_with_ai_confidence' that adds only the advanced AI-generated confidence score
CREATE OR REPLACE TABLE usecases_with_ai_confidence AS
SELECT
  *,
  COALESCE(
    ai_query(
      'databricks-gemma-3-12b',
      REPLACE(
        REPLACE(
          REPLACE(
            REPLACE(
              REPLACE(
                REPLACE(
                  REPLACE(
                    ai_confidence_prompt_advanced,
                    '{context}', COALESCE(context, 'None')
                  ),
                  '{one_liner}', COALESCE(one_liner, 'None')
                ),
                '{next_steps_summary}', COALESCE(next_steps_summary, 'None')
              ),
              '{implementation_notes_summary}', COALESCE(implementation_notes_summary, 'None')
            ),
            '{blockers_summary}', COALESCE(blockers_summary, 'None')
          ),
          '{competition_string}', COALESCE(competition_string, 'None')
        ),
        '{timeline_urgency}', 
        CASE 
          WHEN timeline_urgency IS NOT NULL AND LENGTH(TRIM(timeline_urgency)) > 0 THEN timeline_urgency 
          ELSE 'None' 
        END
      ),
      named_struct('max_tokens', 200, 'temperature', 0.0, 'top_p', 0.1, 'failOnError', false)
    ),
    'CONFIDENCE_SCORE: 0\nLEVEL: Low\nRATIONALE: No information provided.\nDIMENSION_SCORES: Pain=0, Champion=0, ImplementationPlan=0, DecisionProcess=0, Urgency=0, CompetitionAwareness=0, MajorBlockers=0, MeasurableImpact=0'
  ) AS ai_confidence_score_advanced
FROM usecases_with_blockers;

-- Show the first ten advanced AI confidence scores
SELECT ai_confidence_score_advanced
FROM usecases_with_ai_confidence
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE usecases_confidence_scores AS
SELECT
  *,
  -- 1️⃣ Extract confidence score (number 0–100)
  CASE
    WHEN COALESCE(num_of_blockers, 0) > 3 THEN 10
    ELSE CAST(
      regexp_extract(ai_confidence_score_advanced, 'CONFIDENCE_SCORE:\\s*(\\d+)', 1)
      AS INT
    )
  END AS confidence_score,

  -- 2️⃣ Extract confidence level (High / Medium / Low)
  CASE
    WHEN COALESCE(num_of_blockers, 0) > 3 THEN 'Low'
    ELSE regexp_extract(ai_confidence_score_advanced, 'LEVEL:\\s*(High|Medium|Low)', 1)
  END AS confidence_level,

  -- 3️⃣ Extract rationale (until the DIMENSION_SCORES section)
  CASE
    WHEN COALESCE(num_of_blockers, 0) > 3 THEN 'The use case has too many blockers'
    ELSE regexp_extract(ai_confidence_score_advanced, 'RATIONALE:\\s*(.+?)\\s*DIMENSION_SCORES:', 1)
  END AS rationale_for_confidence_score,

  -- 4️⃣ Extract individual dimension scores
  CAST(regexp_extract(ai_confidence_score_advanced, 'Pain=(\\d+)', 1) AS INT) AS pain_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'Champion=(\\d+)', 1) AS INT) AS champion_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'ImplementationPlan=(\\d+)', 1) AS INT) AS implementationplan_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'DecisionProcess=(\\d+)', 1) AS INT) AS decisionprocess_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'Urgency=(\\d+)', 1) AS INT) AS urgency_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'CompetitionAwareness=(\\d+)', 1) AS INT) AS competitionawareness_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'MajorBlockers=(\\d+)', 1) AS INT) AS majorblockers_score,
  CAST(regexp_extract(ai_confidence_score_advanced, 'MeasurableImpact=(\\d+)', 1) AS INT) AS measurableimpact_score

FROM usecases_with_ai_confidence;

SELECT
  confidence_score,
  confidence_level,
  rationale_for_confidence_score,
  pain_score,
  champion_score,
  implementationplan_score,
  decisionprocess_score,
  urgency_score,
  competitionawareness_score,
  majorblockers_score,
  measurableimpact_score
FROM usecases_confidence_scores
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE usecases_confidence_scores_normalized AS
SELECT
  *,
  CASE
    WHEN num_of_blockers > 3 THEN 'Low'
    WHEN confidence_score >= 75 THEN 'High'
    WHEN confidence_score >= 45 THEN 'Medium'
    WHEN confidence_score IS NOT NULL THEN 'Low'
    ELSE 'Not computed'
  END AS confidence_level_normalized

FROM usecases_confidence_scores;

-- Show the first 10 rows with confidence_score and its normalized level
SELECT
  confidence_score,
  confidence_level_normalized
FROM usecases_confidence_scores_normalized
LIMIT 10;

In [0]:
%sql


CREATE OR REPLACE TABLE usecases_with_analysis
AS
SELECT
  *,
  COALESCE(
    ai_query(
      'databricks-gpt-oss-20b',
      CONCAT(
        'You are a Databricks Solutions Architect helping an account team decide the next best action for a customer account and a specific use case.\n',
        'Inputs:\n',
        'Account context: ', COALESCE(context, 'None'), '\n',
        'Use Case: ', COALESCE(one_liner, 'None'), '\n',
        'Next steps summary: ', COALESCE(next_steps_summary, 'None'), '\n',
        'Implementation notes: ', COALESCE(implementation_notes_summary, 'None'), '\n',
        'Blockers: ', COALESCE(blockers_summary, 'None'), '\n',
        'Competition: ', COALESCE(competition_string, 'None'), '\n',
        'MEDDPICC dimension scores: Pain=', COALESCE(CAST(pain_score AS STRING), 'None'),
        ', Champion=', COALESCE(CAST(champion_score AS STRING), 'None'),
        ', ImplementationPlan=', COALESCE(CAST(implementationplan_score AS STRING), 'None'),
        ', DecisionProcess=', COALESCE(CAST(decisionprocess_score AS STRING), 'None'),
        ', Urgency=', COALESCE(CAST(urgency_score AS STRING), 'None'),
        ', CompetitionAwareness=', COALESCE(CAST(competitionawareness_score AS STRING), 'None'),
        ', MajorBlockers=', COALESCE(CAST(majorblockers_score AS STRING), 'None'),
        ', MeasurableImpact=', COALESCE(CAST(measurableimpact_score AS STRING), 'None'), '\n',
        'Overall confidence level: ', COALESCE(confidence_level_normalized, 'None'), '\n\n',
        'Rules:\n',
        '1. Base your recommendation strictly on the evidence provided — do not assume missing information.\n',
        '2. Prioritize actions that maximize the likelihood of a successful go-live and reduce blockers.\n',
        '3. If Pain, Champion, or Implementation Plan is low (<5/10), suggest actions to strengthen these dimensions first.\n',
        '4. If Major Blockers > 5/10, suggest actions to mitigate or remove the blockers.\n',
        '5. If Urgency is high (>7/10), prioritize time-sensitive actions.\n',
        '6. Provide one concise, actionable next step, written as if for the account team to execute within the next week.\n',
        '7. Keep your answer short — 1–2 sentences max.\n',
        '8. Be deterministic: always follow the rules exactly and do not vary wording unnecessarily.\n\n',
        'Return output in this exact format (no extra text):\n\n',
        'NEXT_BEST_ACTION: [Short, actionable recommendation for the account team]\n',
        'RATIONALE: [Brief explanation linking the action to the MEDDPICC scores and blockers, ≤2 sentences]'
      ),
      named_struct('max_tokens', 200, 'temperature', 0.0, 'top_p', 0.1, 'failOnError', false)
    ),
    'NEXT_BEST_ACTION: Review MEDDPICC evidence and address missing information.\nRATIONALE: No sufficient context or scores provided to generate a specific action.'
  ) AS next_best_action_recommendation
FROM usecases_confidence_scores_normalized;

-- Show the first 10 recommendations created
SELECT next_best_action_recommendation
FROM usecases_with_next_best_action
LIMIT 10;

In [0]:
%sql
CREATE OR REPLACE TABLE slippage_categories
USING DELTA
AS SELECT * FROM VALUES
  ('Technical', 'Slippage stems from missing features, technical blockers, or unresolved bugs'),
  ('Business', 'Slippage is due to shifting business priorities, unclear value, or deprioritization'),
  ('Stakeholder', 'Slippage results from lack of stakeholder buy-in, engagement, or sponsor turnover'),
  ('Budget', 'Slippage is caused by funding gaps, delayed approvals, or budget freezes'),
  ('Project Timelines', 'Slippage is driven by unrealistic schedules, missed milestones, or dependencies'),
  ('Data', 'Slippage comes from data issues (access, quality, privacy, or integration)'),
  ('Integration', 'Slippage is due to system connectivity, API, or tooling integration problems'),
  ('Partner', 'Slippage originates from partner-side limitations, bandwidth, or enablement gaps'),
  ('Hyperscaler', 'Slippage stems from hyperscaler constraints (region, quota, roadmap, or policies)'),
  ('Competition', 'Slippage is triggered by market shifts or competitive reprioritization'),
  ('External dependencies', 'Slippage comes from external third parties or customer-side dependencies'),
  ('Other', 'Slippage cause unclear')
AS t(type, description);

In [0]:
%sql
-- Classify use case slippage likelihood and category using AI
CREATE OR REPLACE TABLE usecases_with_slippage AS
SELECT 
  *,
  
  -- Flag use cases likely to slip
  CASE 
    WHEN confidence_level_normalized = 'Medium' THEN true
    ELSE false
  END AS likely_to_slip,

  -- Classify slippage category
  CASE
    WHEN confidence_level_normalized = 'Medium'
    THEN COALESCE(
      ai_classify(
        CONCAT(
          'You are a Databricks Senior Solutions Architect.\n',
          'Classify the PRIMARY reason this use case is likely to slip. Choose exactly ONE from:\n',
          '[Technical, Business, Stakeholder, Budget, Project Timelines, Data, Integration, Partner, Hyperscaler, Competition, Other, External dependencies]\n\n',
          'Decision Rules:\n',
          '- Technical → missing features, blockers, or bugs.\n',
          '- Business → shifting priorities or unclear value.\n',
          '- Stakeholder → low buy-in or sponsor turnover.\n',
          '- Budget → funding or approval delays.\n',
          '- Project Timelines → missed milestones or dependencies.\n',
          '- Data → quality, access, or integration issues.\n',
          '- Integration → tooling or system connectivity issues.\n',
          '- Partner → bandwidth or enablement gaps.\n',
          '- Hyperscaler → regional, quota, or policy constraints.\n',
          '- Competition → market or rival-driven reprioritization.\n',
          '- External dependencies → third-party or customer-side blockers.\n',
          '- Other → none of the above.\n\n',
          'Inputs:\n',
          'Confidence Score Rationale: ', COALESCE(LEFT(rationale_for_confidence_score, 200), 'N/A'), '\n',
          'Next Step: ', COALESCE(LEFT(next_best_action_recommendation, 200), 'N/A')
        ),
        ARRAY(
          'Technical', 'Business', 'Stakeholder', 'Budget', 'Project Timelines', 'Data', 'Integration', 'Partner', 'Hyperscaler', 'Competition', 'Other', 'External dependencies'
        )
      ),
      'Unable to determine slippage category'
    )
    ELSE 'N/A'
  END AS slippage_category

FROM usecases_with_analysis;

-- Show the first 10 rows with only the created field
SELECT rationale_for_confidence_score, next_best_action_recommendation, slippage_category
FROM usecases_with_slippage
LIMIT 10;

In [0]:
%sql 
CREATE OR REPLACE TABLE acceleration_categories 
USING DELTA
AS SELECT * FROM VALUES
  ('Implementation Planning', 'Acceleration comes from clarifying the WHAT/WHEN/WHO (critical path, milestones, owners)'),
  ('Resource Allocation', 'The plan exists but we need people/tools/compute NOW'),
  ('Stakeholder Engagement', 'Acceleration is through stakeholder alignment, exec cadence, or clearing decisions'),
  ('Technical Optimization', 'Acceleration is via performance, architecture, or integration tuning'),
  ('Innovation and Technology Enablement', 'Acceleration is via templates, automation, or accelerators (including AI assistants)'),
  ('Governance and Support', 'Acceleration is via operating rhythm (steering, dashboards, SLAs, support processes)'),
  ('Capability Development', 'Acceleration is via training/upskilling/knowledge transfer'),
  ('Partnership Leverage', 'Acceleration comes when a partner''s expertise/assets are the key lever'),
  ('Funding', 'Acceleration n is due to securing or expediting funding, budget approvals, or financial resources'),
  ('Other', 'Unclear or does not fit any other acceleration motion')
AS t(type, description);

In [0]:
%sql
-- Classify acceleration categories using AI
CREATE OR REPLACE TABLE  usecases_with_acceleration AS
SELECT 
  *,
  CASE 
    WHEN confidence_level_normalized = 'Medium' THEN true
    When confidence_level_normalized = 'High' THEN true
    ELSE false
  END AS can_be_accelerated,
  
  CASE
    WHEN confidence_level_normalized in ('Medium','High') THEN 
      COALESCE(
        ai_classify(
          CONCAT(
            'You are a Databricks Senior Solutions Architect.\n',
            'Classify the PRIMARY reason this use case could be accelerated.\n',
            'Choose exactly ONE label from:\n',
            '[Implementation Planning, Resource Allocation, Stakeholder Engagement, Technical Optimization, Partnership Leverage, Governance and Support, Innovation and Technology Enablement, Capability Development, Funding, Other]\n\n',
            'Decision rules:\n',
            '- Implementation Planning → clarifying the WHAT/WHEN/WHO (critical path, milestones, owners).\n',
            '- Resource Allocation → people/tools/compute NOW.\n',
            '- Stakeholder Engagement → alignment, cadence, or decisions.\n',
            '- Technical Optimization → performance, architecture, integration tuning.\n',
            '- Innovation and Technology Enablement → templates, automation, AI accelerators.\n',
            '- Governance and Support → steering, dashboards, SLAs, support.\n',
            '- Capability Development → training, upskilling, knowledge transfer.\n',
            '- Partnership Leverage → partner expertise/assets as key lever.\n',
            '- Funding → securing or expediting budget, approvals, resources.\n',
            '- Other → none of the above.\n\n',
            'Return exactly one of the labels above — no punctuation or explanation.\n\n',
            'Context:\n',
            'Confidence Score Rationale: ', COALESCE(LEFT(rationale_for_confidence_score, 200), 'N/A'), '\n',
            'Next Step: ', COALESCE(LEFT(next_best_action_recommendation, 200), 'N/A')
          ),
          ARRAY(
            'Implementation Planning', 'Resource Allocation', 'Stakeholder Engagement', 'Technical Optimization', 
            'Partnership Leverage', 'Governance and Support', 'Innovation and Technology Enablement', 
            'Capability Development', 'Funding', 'Other'
          )
        ),
        'Unable to determine acceleration category'
      )
    ELSE 'N/A'
  END AS accel_category
FROM usecases_with_slippage;


-- Show the first 10 rows with only the created field
SELECT rationale_for_confidence_score, next_best_action_recommendation, accel_category
FROM usecases_with_acceleration
LIMIT 10;


In [0]:
%sql

-- Create the final table with all AI-enriched fields
CREATE OR REPLACE TABLE pipelineiq
USING DELTA
TBLPROPERTIES ('delta.minReaderVersion'='2', 'delta.minWriterVersion'='5')
COMMENT 'PipelineIQ: Use case pipeline with AI-enriched fields.'
AS SELECT * FROM usecases_with_acceleration;

-- Display the first 10 lines of pipelineiq
SELECT * FROM pipelineiq LIMIT 10;

In [0]:
%sql
-- Add column comments for new AI-generated fields
ALTER TABLE pipelineiq ALTER COLUMN context COMMENT 'AI-generated summary of BDR context and business impact for the use case';
ALTER TABLE pipelineiq ALTER COLUMN one_liner COMMENT 'AI-generated concise summary of the use case';
ALTER TABLE pipelineiq ALTER COLUMN next_steps_summary COMMENT 'AI-generated summary of next steps for the use case';
ALTER TABLE pipelineiq ALTER COLUMN implementation_notes_summary COMMENT 'AI-generated summary of implementation notes';
ALTER TABLE pipelineiq ALTER COLUMN blockers_summary COMMENT 'AI-generated summary of blockers';
ALTER TABLE pipelineiq ALTER COLUMN usecase_type COMMENT 'AI-classified technical use case category';
ALTER TABLE pipelineiq ALTER COLUMN business_usecase_type COMMENT 'AI-generated business use case label (max 5 words)';
ALTER TABLE pipelineiq ALTER COLUMN confidence_score COMMENT 'AI-extracted confidence score (0-100) for successful go-live';
ALTER TABLE pipelineiq ALTER COLUMN confidence_level COMMENT 'AI-extracted confidence level (High, Medium, Low)';
ALTER TABLE pipelineiq ALTER COLUMN rationale_for_confidence_score COMMENT 'AI-extracted rationale for confidence score';
ALTER TABLE pipelineiq ALTER COLUMN pain_score COMMENT 'AI-extracted MEDDPICC Pain dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN champion_score COMMENT 'AI-extracted MEDDPICC Champion dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN implementationplan_score COMMENT 'AI-extracted MEDDPICC Implementation Plan dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN decisionprocess_score COMMENT 'AI-extracted MEDDPICC Decision Process dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN urgency_score COMMENT 'AI-extracted MEDDPICC Urgency dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN competitionawareness_score COMMENT 'AI-extracted MEDDPICC Competition Awareness dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN majorblockers_score COMMENT 'AI-extracted MEDDPICC Major Blockers dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN measurableimpact_score COMMENT 'AI-extracted MEDDPICC Measurable Impact dimension score (0-10)';
ALTER TABLE pipelineiq ALTER COLUMN confidence_level_normalized COMMENT 'Normalized confidence level (High, Medium, Low, Not computed)';
ALTER TABLE pipelineiq ALTER COLUMN next_best_action_recommendation COMMENT 'AI-generated next best action recommendation for account team';
ALTER TABLE pipelineiq ALTER COLUMN likely_to_slip COMMENT 'Flag if use case is likely to slip (true/false)';
ALTER TABLE pipelineiq ALTER COLUMN slippage_category COMMENT 'AI-classified primary reason for slippage';
ALTER TABLE pipelineiq ALTER COLUMN can_be_accelerated COMMENT 'Flag if use case can be accelerated (true/false)';
ALTER TABLE pipelineiq ALTER COLUMN accel_category COMMENT 'AI-classified best acceleration lever';
ALTER TABLE pipelineiq ALTER COLUMN is_stale COMMENT 'Flag indicating if the use case is stale (not updated in 14+ days)';

In [0]:
%sql
ALTER TABLE pipelineiq SET TBLPROPERTIES (
  'ai.model.version' = 'databricks-gpt-oss-20b',
  'ai.generation.timestamp' = '2024-10-28',
  'ai.prompt.version' = 'v1.3'
);

In [0]:
%sql
-- Create final view with all requested columns
CREATE OR REPLACE VIEW pipelineiq_view AS
SELECT 
  -- Original fields
  snapshot_date,
  field_manager,
  account_name,
  usecase_id,
  usecase_nhttps://adb-2548836972759138.18.azuredatabricks.net/dashboardsv3/3542653191289487/published?o=2548836972759138$0ame,
  last_modified_date,
  account_executive,
  account_solution_architect,
  stage,
  target_cloud,
  primary_competitor,
  target_live_date,
  target_onboarding_date,
  estimated_monthly_dollar_dbus,
  usecase_description,
  demand_plan_stage_next_steps,
  implementation_notes,
  sales_region,
  sales_subregion_level_1,
  sales_subregion_level_2,
  sales_subregion_level_3,
  sales_subregion_level_4,
  sales_manager,
  sales_leader,
  field_leader,
  use_case_product,
  blockers_last_modified_date,
  sdr_bdr_notes,
  business_impact,
  business_outcome_description,
  ae_authority,
  ae_money_budget,
  business_unit,
  num_of_blockers,
  last_modified_blocker_category,
  last_modified_blocker_comment,
  competition_string,
  is_stale, 
  usecase_description,
  demand_plan_stage_next_steps,
  implementation_notes,
  -- AI-generated fields
  context,
  soundbyte,
  one_liner,
  next_steps_summary,
  implementation_notes_summary,
  blockers_summary,
  usecase_type,
  business_usecase_type,
  ai_confidence_score_advanced,
  confidence_score,
  confidence_level,
  rationale_for_confidence_score,
  pain_score,
  champion_score,
  implementationplan_score,
  decisionprocess_score,
  urgency_score,
  competitionawareness_score,
  majorblockers_score,
  measurableimpact_score,
  confidence_level_normalized,
  next_best_action_recommendation,
  likely_to_slip,
  slippage_category,
  can_be_accelerated,
  accel_category,
  -- Set vertical to 'not specified' if null
  COALESCE(vertical, 'Not specified') AS vertical
FROM pipelineiq;

-- Display the first 10 lines of the view
SELECT * FROM pipelineiq_view LIMIT 10;

In [0]:
%sql
COMMENT ON VIEW pipelineiq_view IS 'Business-facing view of PipelineIQ with cleaned recommendation labels, normalized verticals, and presentation-ready AI enrichment.';
ALTER VIEW pipelineiq_view SET TBLPROPERTIES (
  'ai.pipeline.version' = 'v1.3',
  'ai.model' = 'databricks-gpt-oss-20b',
  'ai.view.purpose' = 'Executive insights and reporting'
);